In [11]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import os
import pickle
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import time
%matplotlib inline

import our libaries

In [12]:
import preprocess as pp
import vectorizer as vr
import cluster as clst
import analyze as anlz

In [13]:
t = time.localtime()
timestamp = time.strftime('%b_%d_%Y_%H%M', t)
np.random.seed(4)
if not os.path.exists('results'):
    os.mkdir('results')
if not os.path.exists('results/' + timestamp):
    os.mkdir('results/' + timestamp)
with open('results/' + timestamp +'/log.txt','w') as ff:
    with open('main.py', 'r') as fp:
        ff.write(fp.read())

parmeters

In [14]:
min_df = 1e-4#1e-10
max_df = 0.05#1
k = 100
max_num = 1000#len(emails)

In [15]:
newsgroups_train = fetch_20newsgroups(subset='train')
labels = newsgroups_train.target
labels_names = newsgroups_train.target_names
#TODO: consider take only body
emails = newsgroups_train.data
emails = emails[:max_num]

preprocess

In [16]:
emails = [pp.clean_text(e) for e in emails]

vectorizing

In [17]:
cv,tokenized_emails = vr.tokenizer(emails,min_df,max_df)
#{k: v for k, v in zip(cv.get_feature_names(), sample_vec.toarray()[0]) if v > 0}
# to get vocabulary names use: cv.get_feature_names()
voc_names = cv.get_feature_names()
#to get BOW for each mail use: tokenized_emails.toarray()
emails_representation = tokenized_emails.toarray()

NameError: name 'sample_vec' is not defined

In [ ]:
tf_idf = vr.tf_idf(emails,min_df,max_df)
emails_representation = tf_idf.toarray()

In [ ]:
emails_representation = vr.BOW_w2v(emails)

clustering

In [ ]:
print('compute kmeans clusters. vector shape:{}'.format(emails_representation.shape[1]))
clusters,kmns_class = clst.kmeans_cosine_dist(w2v,k)

In [ ]:
clusters,kmns_class = clst.kmeans(w2v,k)

In [ ]:
(clusters, _) = clst.hirarchical(w2v, k,aff='euclidean',link='ward')

analayze

In [ ]:
number_of_labels = 20
random_clst = np.random.randint(0, k, max_num)
conf_mat_rand = confusion_matrix(labels[:max_num], random_clst)
conf_mat = confusion_matrix(labels[:max_num], clusters)


plt.figure(1)
plt.imshow(conf_mat)
plt.savefig('results/' + timestamp + '/conf_mat.png')


ret_rand = anlz.evaluate_many2one(conf_mat_rand,number_of_labels)
ret = anlz.evaluate_many2one(conf_mat, number_of_labels)
print('acc: {} (random={})'.format(np.mean(ret),np.mean(ret_rand)))

plt.figure(2)
plt.plot(range(1, 1 + number_of_labels), ret_rand, '*')
plt.plot(range(1, 1 + number_of_labels), ret, '*r')
plt.ylim([0, 1])
plt.savefig('results/' + timestamp + '/acc.png')


with open('results/' + timestamp + '/kmeans.pkl','wb') as wf:
    pickle.dump(conf_mat,wf)
